In [28]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [29]:
print("Number of characters in dataset: ", len(text))

Number of characters in dataset:  1115394


In [30]:
# First 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [31]:
# Possible characters the model can see/emit
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [32]:
# Tokenization
# goes from characters to ints, simply a lookup table since its simple enough
# several types of schemes eg. SentencePiece, TikToken

stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("How are you doing today?")))

# OpenAI scheme, sequences are much shorter since they aren't character-level
import tiktoken
enc = tiktoken.get_encoding('gpt2')
print(enc.encode("How are you doing today?"))
print(enc.decode(enc.encode("How are you doing today?")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
How are you doing today?
[2437, 389, 345, 1804, 1909, 30]
How are you doing today?


In [33]:
# Tokenize the entire dataset
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print("Dataset size and shape: ", data.shape, data.dtype)
print(data[:1000])

# train/test split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print("Train/Val split, ", train_data.shape, val_data.shape)

Dataset size and shape:  torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  

In [34]:
# Need to build a data loader
block_size = 8 # context length

train_data[:block_size+1] # represents multiple pieces of data, at each position n-1 examples

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [35]:
# Essentially the "time" dimension
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is: {target}")

When input is tensor([18]) the target is: 47
When input is tensor([18, 47]) the target is: 56
When input is tensor([18, 47, 56]) the target is: 57
When input is tensor([18, 47, 56, 57]) the target is: 58
When input is tensor([18, 47, 56, 57, 58]) the target is: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [36]:
# introduce the batch dimension
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(xb.shape, yb.shape)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target is: {target}")

torch.Size([4, 8]) torch.Size([4, 8])
When input is [24] the target is: 43
When input is [24, 43] the target is: 58
When input is [24, 43, 58] the target is: 5
When input is [24, 43, 58, 5] the target is: 57
When input is [24, 43, 58, 5, 57] the target is: 1
When input is [24, 43, 58, 5, 57, 1] the target is: 46
When input is [24, 43, 58, 5, 57, 1, 46] the target is: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
When input is [44] the target is: 53
When input is [44, 53] the target is: 56
When input is [44, 53, 56] the target is: 1
When input is [44, 53, 56, 1] the target is: 58
When input is [44, 53, 56, 1, 58] the target is: 46
When input is [44, 53, 56, 1, 58, 46] the target is: 39
When input is [44, 53, 56, 1, 58, 46, 39] the target is: 58
When input is [44, 53, 56, 1, 58, 46, 39, 58] the target is: 1
When input is [52] the target is: 58
When input is [52, 58] the target is: 1
When input is [52, 58, 1] the target is: 58
When input is [52, 58, 1, 58] the target i

In [55]:
# simplest nn: bigram language model

import torch.nn as nn
from torch.nn import functional as F 

torch.manual_seed(1337)

# P(token_t | token_{t-1})
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()

        # each token reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx/targets shape (Batch, Time), logits (Batch, Time, Channels) eg (4, 8, 65)
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # stack the batch and time dimensions


            # do same to targets
            targets = targets.view(B*T)

            # Wants (Batch, Channels, Time)
            loss = F.cross_entropy(logits, targets)


        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (Batch, Time) of indicies cur context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)

            # Last time step
            logits = logits[:, -1, :] # (Batch, Channels)
            probs = F.softmax(logits, dim=-1) # (Batch, Channels)
            idx_next = torch.multinomial(probs, num_samples=1) #(Batch, 1)
            idx = torch.cat((idx, idx_next), dim=1) #(Batch, Time + 1)
        return idx # (Batch, Time + max_new_tokens)
    torch.manual_seed(1337)

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([256, 65])
tensor(4.6975, grad_fn=<NllLossBackward0>)


In [56]:
# Train the model
eval_iters = 300
batch_size = 32
max_iters = 20000

m = BigramLanguageModel(vocab_size)
opt = torch.optim.AdamW(m.parameters(), lr=1e-3)

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            logits, loss = m(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [57]:
for step in range(max_iters):
    if step % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch1('train')

    logits, loss = m(xb, yb)
    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()


step 0: train loss 4.6179, val loss 4.6223
step 300: train loss 4.2762, val loss 4.2865
step 600: train loss 3.9850, val loss 3.9954
step 900: train loss 3.7328, val loss 3.7352
step 1200: train loss 3.5061, val loss 3.5194
step 1500: train loss 3.3192, val loss 3.3296
step 1800: train loss 3.1620, val loss 3.1783
step 2100: train loss 3.0286, val loss 3.0486
step 2400: train loss 2.9299, val loss 2.9436
step 2700: train loss 2.8456, val loss 2.8602
step 3000: train loss 2.7746, val loss 2.7877
step 3300: train loss 2.7196, val loss 2.7379
step 3600: train loss 2.6857, val loss 2.6893
step 3900: train loss 2.6446, val loss 2.6519
step 4200: train loss 2.6153, val loss 2.6225
step 4500: train loss 2.5896, val loss 2.5963
step 4800: train loss 2.5686, val loss 2.5847
step 5100: train loss 2.5612, val loss 2.5666
step 5400: train loss 2.5375, val loss 2.5592
step 5700: train loss 2.5340, val loss 2.5506
step 6000: train loss 2.5112, val loss 2.5377
step 6300: train loss 2.5200, val loss 2

In [77]:
def generate_text(model, length):
    context = torch.zeros((1,1), dtype=torch.long)
    print(decode(model.generate(context, max_new_tokens=length)[0].tolist()))

generate_text(m, 300)

IndexError: index out of range in self

In [67]:
# Mathematical Trick used in self-attention
torch.manual_seed(1337)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

# We want to couple each token, only communicate with the tokens in the past
# Simplest thing is to just do an average of all previous tokens before me

# want x[b, t] = mean_{i<=t} x[b, i]
x_bag_of_words = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        x_bag_of_words[b, t] = torch.mean(xprev, 0) #(1, C)

# print(x_bag_of_words[0])

# How to do this efficiently with matrix multiplication?
# torch.tril gets the lower triangular mat, exactly what we need to do this "causal" average
o = torch.ones((T, T)) # <- these would not be ones, but learnable weights
weight = torch.tril(o)
weight = weight / torch.sum(weight, 1, keepdim=True)

xbow = weight @ x
# print(xbow[0])
# torch.allclose(x_bag_of_words[0], xbow[0])

# Another way to re-write with softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow2 = wei @ x
# print(xbow2[0])

# Last way, self-attention
# every token emits a Query/Key vector, affinities are dot products Q.dot(K)

# Single head self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)
wei *= head_size ** -0.5 # Scale so softmax is more diffuse, try to get them to be unit variance

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x) 
out = wei @ v

print(out.shape)

torch.Size([4, 8, 16])


In [103]:
vocab_size = 65
n_embed = 384

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()

        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    
    def forward(self, x):
        B, T, C = x.shape # (batch, time, context_length)

        k = self.key(x)
        q = self.query(x)

        w = q @ k.transpose(-2, -1) 
        w *= head_size ** (-0.5)
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (batch, time, time)
        w = F.softmax(w, dim=-1)
        w = self.dropout(w)

        v = self.value(x)
        return w @ v


class BiGramTransformer(nn.Module):
    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.sa_head = Head(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx/targets (B, T)
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T)) # (T, C)
        x = tok_emb + pos_emb
        x = self.sa_head(x) # apply one head of self-attention (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, vocab_size)

            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (Batch, Time) of indicies cur context
        for _ in range(max_new_tokens):
            idx_crop = idx[:, -block_size:]
            logits, loss = self(idx_crop)

            # Last time step
            logits = logits[:, -1, :] # (Batch, Channels)
            probs = F.softmax(logits, dim=-1) # (Batch, Channels)
            idx_next = torch.multinomial(probs, num_samples=1) #(Batch, 1)
            idx = torch.cat((idx, idx_next), dim=1) #(Batch, Time + 1)
        return idx # (Batch, Time + max_new_tokens)

In [109]:

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            logits, loss = m(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

def train(m):
    # Train the model
    batch_size = 64
    block_size = 256
    max_iters = 5000
    eval_iters = 300
    eval_interval = 500
    n_embed = 384
    dropout = 0.2

    opt = torch.optim.AdamW(m.parameters(), lr=3e-4)

    for step in range(max_iters):
        if step % eval_interval == 0:
            losses = estimate_loss()
            print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        xb, yb = get_batch('train')

        logits, loss = m(xb, yb)
        opt.zero_grad(set_to_none=True)
        loss.backward()
        opt.step()


In [105]:
generate_text(m, 300)


And the villine and yet and plooches; I word son;
And my fairth.
Mere:
Thand: bast
A cond,
Far mence aretmoush or a cy day print.
Den,
Det befornow,
Than Jeat bandy ursell in uth his obmey,
As befothy,
Thust
Bet in
Thell seeccas;
Yout gentserfacionem of them verew
Which And sopeds repereamash,
Dad s


In [110]:
class LayerNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self, x):
        # batch norm would be normalizing the batch, but here we do per element
        x_mean = x.mean(1, keepdim=True)
        x_var = x.var(1, keepdim=True)
        
        # normalize variance 
        x_hat = (x - x_mean) / torch.sqrt(x_var + self.eps) 
        self.out = self.gamma * x_hat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]



class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        self_attention = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(self_attention))

class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, n_embed * 4), # inner space is higher dimension per paper
            nn.ReLU(),
            nn.Linear(n_embed * 4, n_embed),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    # Basically groups attention with feedforward computation

    def __init__(self, n_embed, n_head):
        super().__init__()

        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        # Add residual connections to stabilize training
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class Transformer(nn.Module):
    def __init__(self, n_layer, n_head):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx/targets (B, T)
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T)) # (T, C)
        x = tok_emb + pos_emb
        x = self.blocks(x) # does this per token, ones communcation has happened from attention
        x = self.ln_f (x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, vocab_size)

            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (Batch, Time) of indicies cur context
        for _ in range(max_new_tokens):
            idx_crop = idx[:, -block_size:]
            logits, loss = self(idx_crop)

            # Last time step
            logits = logits[:, -1, :] # (Batch, Channels)
            probs = F.softmax(logits, dim=-1) # (Batch, Channels)
            idx_next = torch.multinomial(probs, num_samples=1) #(Batch, 1)
            idx = torch.cat((idx, idx_next), dim=1) #(Batch, Time + 1)
        return idx # (Batch, Time + max_new_tokens)

In [111]:
m = Transformer(6, 6)
train(m)

open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

step 0: train loss 4.3176, val loss 4.3164
step 500: train loss 2.1459, val loss 2.1935
step 1000: train loss 2.0369, val loss 2.0988
step 1500: train loss 1.9596, val loss 2.0446
step 2000: train loss 1.9167, val loss 2.0322
step 2500: train loss 1.8811, val loss 2.0100
step 3000: train loss 1.8569, val loss 1.9930
step 3500: train loss 1.8323, val loss 1.9732
step 4000: train loss 1.8215, val loss 1.9726
step 4500: train loss 1.8102, val loss 1.9579


10001

In [112]:
generate_text(m, 300)



ROMEO:
Breath my goods,
Whow brance;
Warwixks I mine to that coulding
Btrance!

Third you stingly comforfrie, I cickinge
Withoursed is rised once; the suchsinks crastion.
A everia friet,
You. Thought of in Gods if stroud,
And vown at, bost
On you spired the soul,
Edward you our sist you, carvew, an
